In [2]:
import pandas as pd
import dash
from dash import dcc
from dash import html

import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
import webbrowser
from threading import Timer





In [ ]:
#Load Data 
disaster_stock_data= pd.read_pickle("Users/jan/Techlabs_WS_Team3/02_code/0_preprocess/disaster_stock_data.pkl")
nikkei_companies= pd.read_pickle("Users/jan/Techlabs_WS_Team3/02_code/0_preprocess/nikkei_companies.pkl")

In [11]:
disaster_stock_data['state'] = disaster_stock_data['state'].fillna("").astype(str)

In [ ]:
# 📌 Dash App erstellen
app = dash.Dash(__name__)

# 📌 Layout mit hellem Design & Filtern über der Map
app.layout = html.Div(
    style={"backgroundColor": "white", "color": "black", "padding": "20px", "minHeight": "100vh"},
    children=[
        html.H1(children='Stock_Market Analysis',
                style={'textAlign': 'left', "color": "#8ABF8B"}),

        # 🔽 Filter als Dropdowns über der Karte
# 🔽 Filter als Dropdowns über der Karte mit individuellen Überschriften
html.Div(
    style={"display": "flex", "justifyContent": "left", "gap": "20px", "marginBottom": "20px"},
    children=[
        html.Div([
            html.Label("Year", style={"fontSize": "15px", "color": "black"}),
            dcc.Dropdown(
                id='dropdown-selection',
                options=[{'label': date, 'value': date} for date in disaster_stock_data.year.unique()],
                value=disaster_stock_data.year.unique()[0],
                clearable=False,
                style={"backgroundColor": "white", "color": "black", "width": "250px"},
                placeholder="Wähle ein Datum",
            )
        ], style={"display": "flex", "flexDirection": "column"}),

        html.Div([
            html.Label("State", style={"fontSize": "15px", "color": "black"}),
            dcc.Dropdown(
                id='state-selection',
                options=[{'label': state, 'value': state} for state in disaster_stock_data.state.unique()],
                value=disaster_stock_data.state.unique()[0],
                clearable=False,
                style={"backgroundColor": "white", "color": "black", "width": "250px"},
                placeholder="Wähle einen State",
            )
        ], style={"display": "flex", "flexDirection": "column"}),

        html.Div([
            html.Label("Industry", style={"fontSize": "15px", "color": "black"}),
            dcc.Dropdown(
                id='industry-selection',
                options=[{'label': industry, 'value': industry} for industry in nikkei_companies.Industry.unique()],
                value=nikkei_companies.Industry.unique()[0],
                clearable=False,
                style={"backgroundColor": "white", "color": "black", "width": "250px"},
                placeholder="Wähle eine Industry",
            )
        ], style={"display": "flex", "flexDirection": "column"}),
    ],
),


        # 🌍 Map Bereich & Rechte Seite
        html.Div(
            id="main-container",
            style={"display": "flex", "gap": "20px", "height": "80vh"},  # Höhe setzen
            children=[
                html.Div(  # Map-Bereich (nimmt 50% ein)
                    id="map-container",
                    style={"flex": "0.5"},
                    children=[dcc.Graph(id='graph-content', style={"height": "100%"})],
                ),
                html.Div(  # Rechte Seite (nimmt 50% ein)
                    id="right-panel",
                    style={"flex": "0.5", "backgroundColor": "#f8f9fa", "padding": "20px", "borderRadius": "10px"},
                    children=[
                        html.H3("Weitere Informationen"),
                        html.P("Hier kannst du zusätzliche Inhalte einfügen."),
                    ],
                ),
            ],
        ),
    ]
)

# 🎯 Callback: Aktualisiert die Karte basierend auf den Filtern
@app.callback(
    Output('graph-content', 'figure'),
    [Input('dropdown-selection', 'value'),
     Input('state-selection', 'value'),
     Input('industry-selection', 'value')]
)
def update_map(selected_date, selected_state, selected_industry):
    # 🔍 Daten filtern
    filtered_data = disaster_stock_data[
        (disaster_stock_data['year'] == selected_date) &
        (disaster_stock_data['state'] == selected_state)
    ]
    # Unternehmens-Daten filtern
    filtered_companies = nikkei_companies[nikkei_companies['Industry'] == selected_industry]

    # 📊 Falls keine Daten vorhanden sind, leere Karte zurückgeben
    if filtered_data.empty and filtered_companies.empty:
        return go.Figure()

    # 🌍 Density Map mit CARTO-Style
    fig = px.density_mapbox(
        filtered_data,
        lat="latitude",
        lon="longitude",
        z="magnitudo",
        hover_name="place",
        radius=10,  # Dichte-Radius für bessere Sichtbarkeit
        zoom=5,
        mapbox_style="carto-positron",  # Setzt CARTO als Kartenstil
    )

    # Unternehmen als zusätzliche Marker hinzufügen
    if not filtered_companies.empty:
        fig.add_trace(go.Scattermapbox(
            lat=filtered_companies["latitude"].tolist(),
            lon=filtered_companies["longitude"].tolist(),
            mode="markers",
            marker=dict(size=15, color="cyan"),
            hovertext=filtered_companies["Company_name"],
            hoverinfo="text",
        
        ))

    # 🖌️ Styling & Design
    fig.update_layout(
        margin=dict(l=10, r=10, t=20, b=10, pad=5),
        plot_bgcolor="white",
        paper_bgcolor="white",
        clickmode="event+select",
        hovermode="closest",
        coloraxis_showscale=False,
        showlegend=False,
        mapbox=dict(
            accesstoken="pk.eyJ1IjoidGVjaGxhYnMzIiwiYSI6ImNtNzBib2xyczAwZHoycnBiM2hxZ24zcngifQ.AaSHNEfc-cnR4uDdEO4gsw",
            bearing=10,
            center=dict(
                lat=filtered_data.latitude.mean() if not filtered_data.empty else 50,
                lon=filtered_data.longitude.mean() if not filtered_data.empty else 10
            ),
            pitch=5,
            zoom=5,
        ),
    )

    return fig

# 🌍 Automatisches Öffnen des Browsers
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

# 🚀 Starte die App im Browser
if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True)

/var/folders/xm/s7wz86kd7szcj2pdp0nfk8mh0000gn/T/ipykernel_56469/2299222923.py:99: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/xm/s7wz86kd7szcj2pdp0nfk8mh0000gn/T/ipykernel_56469/2299222923.py:112: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

